In [1]:
# required imports

!pip install openpyxl
import openpyxl

import xlrd
import random
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

In [2]:
# Read the first sheet from the Excel File

file_name = 'Yearwise.xlsx'
# Use openpyxl to open .xlsx files
wb = openpyxl.load_workbook(file_name)
# Access the desired sheet by name or index (e.g., 0 for the first sheet)
sheet = wb.active # or wb['Sheet1'] if the sheet name is 'Sheet1'


FileNotFoundError: [Errno 2] No such file or directory: 'Yearwise.xlsx'

In [ ]:
# Some pre-processing to prepare the DataFrame outof the excel file

prefix = 'Cases Reported during the Year (CR) Under Crime against Women in '
suffix = 'During 2001-2015'
state_name = None
key = None
info_list = []
info = dict()

# Get the maximum number of rows and columns
max_row = sheet.max_row
max_column = sheet.max_column


for i in range(1, max_row + 1): # Iterate through rows starting from index 1
    numbers = list()
    for j in range(1, max_column + 1): # Iterate through columns starting from index 1
        value = sheet.cell(row=i, column=j).value # Get cell value using openpyxl syntax
        if type(value) is str and not value.split():
            continue
        if value == 'SL':
            break
        if j == 1 and type(value) is str and 'Cases Reported during the Year' in value:
            state_name = value.replace(prefix, '').replace(suffix, '')
            info['state_name'] = state_name
            break
        if j == 2 and type(value) is str and value.split(): # Changed to column 2 for key
            key = value
            info[key] = None
        if type(value) is float or type(value) is int: # Include int type
            numbers.append(value)
    if numbers:
        info[key] = numbers[1:]
        n_keys = len(list(info.keys()))
        if n_keys == 16:
            info_list.append(info)
            info = dict()

keys = list(info_list[0].keys())
columns = keys.copy()
columns.insert(1,'Year')

final_list = list()

for il in info_list:
    year = 2001
    for i in range(15):
        temp_list = list()
        for k in keys[1:]:
            temp_list.append(il.get(k)[i])
        temp_list.insert(0, il.get('state_name').strip())
        temp_list.insert(1, year)
        year += 1
        final_list.append(temp_list)

In [ ]:
# preview the prepared DataFrame

df = pd.DataFrame(final_list, columns=columns)
df.head()

In [ ]:
# Keeping only the required columns

state_wise_total_crimes = df[['state_name', 'Total Crimes against Women', 'Year']]
data_for_map = state_wise_total_crimes[state_wise_total_crimes['Year']==2015]
data_for_map.head()

In [ ]:
# reading the state wise shapefile of India in a GeoDataFrame and preview it

fp = "Igismap/Indian_States.shp"
map_df = gpd.read_file(fp)
map_df.head()

In [ ]:
# Correct spellings of states from out dataframe to match those of GeoDataframe
# I found these 4 names manually

data_for_map['state_name'].iloc[29]  = 'Andaman & Nicobar Island'
data_for_map['state_name'].iloc[1] = 'Arunanchal Pradesh'
data_for_map['state_name'].iloc[31] = 'Dadara & Nagar Havelli'
data_for_map['state_name'].iloc[33] = 'NCT of Delhi'

In [ ]:
# Plot the default map

map_df.plot()

In [ ]:
# Join both the DataFrames by state names

merged = map_df.set_index('st_nm').join(data_for_map.set_index('state_name'))
merged.head()

In [ ]:
# NA check

merged.isna().sum()

In [ ]:
# Summary to get the max and min value

merged.describe()

In [ ]:
# create figure and axes for Matplotlib and set the title
fig, ax = plt.subplots(1, figsize=(10, 6))
ax.axis('off')
ax.set_title('State Wise Crime against women in India in 2015', fontdict={'fontsize': '25', 'fontweight' : '3'})

# plot the figure
merged.plot(column='Total Crimes against Women', cmap='YlOrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

In [ ]:
# We save the output as a PNG image

fig.savefig("State_wise.png", dpi=100)

In [ ]:
# reading the district wise shapefile of India in a GeoDataFrame and preview it

fp = "gadm36_IND_shp/gadm36_IND_2.shp"
map_df = gpd.read_file(fp)
map_df.head()

In [ ]:
# keep required columns and plot default map

map_df = map_df[['NAME_2', 'geometry']]
map_df.plot()

In [ ]:
# read the district wise excel file and preview it

district_wise = pd.read_excel('district_wise.xls')
district_wise.head()

In [ ]:
# keeping only the required columns

district_wise = district_wise[['District/ Area', 'Total Crimes against Women']]

In [ ]:
# Join both the DataFrames by district names

merged = map_df.set_index('NAME_2').join(district_wise.set_index('District/ Area'))
merged.head()

In [ ]:
# NA Check

merged.isna().sum()

##### Seems like there are around 185 district names that are either mispelled or missing
##### I am not going to do correct these manually instead we will impute them with the mean (even though that will make our data incorrect but the propose of this notebook is to show you how to use GeoPandas to plot District wise stats on the Map of India)

In [ ]:
# Impute missing values by mean

merged['Total Crimes against Women'].fillna(merged['Total Crimes against Women'].mean(), inplace=True)
merged.isna().sum()

In [ ]:
# summary

merged.describe()

In [ ]:
# create figure and axes for Matplotlib and set the title
fig, ax = plt.subplots(1, figsize=(10, 6))
ax.axis('off')
ax.set_title('District Wise Crime against women in India in 2015', fontdict={'fontsize': '25', 'fontweight' : '3'})

# plot the figure
merged.plot(column='Total Crimes against Women', cmap='YlOrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

In [ ]:
# We save the output as a PNG image

fig.savefig("District_wise.png", dpi=100)

In [ ]:
# reading the district wise shapefile of India in a GeoDataFrame and preview it

fp = "gadm36_IND_shp/gadm36_IND_2.shp"
map_df = gpd.read_file(fp)
map_df.head()

In [ ]:
map_df = map_df[['NAME_1', 'NAME_2', 'geometry']]
map_df = map_df[map_df['NAME_1']=='Maharashtra']
map_df.plot()

In [ ]:
district_wise = pd.read_excel('district_wise.xls')
district_wise.head()

In [ ]:
# keeping only the required columns

district_wise = district_wise[['State/ UT', 'District/ Area', 'Total Crimes against Women']]
district_wise = district_wise[district_wise['State/ UT']=='Maharashtra']
district_wise.head()

In [ ]:
# Correct spellings of states from out dataframe to match those of GeoDataframe
# I corrected these names manually

district_wise['District/ Area'].iloc[0] = 'Ahmadnagar'


# 1449 is the sum of Amravati Commr and Amravati Rural from the district wise dataframe
district_wise = pd.concat([district_wise, pd.DataFrame([{'State/ UT': 'Maharashtra', 'District/ Area':'Amravati','Total Crimes against Women':1449.0}])], ignore_index=True)

# 1449 is the sum of Aurangabad Commr and Aurangabad Rural from the district wise Dataframe
# same is done for rest of the cities
district_wise = pd.concat([district_wise, pd.DataFrame([{'State/ UT': 'Maharashtra', 'District/ Area':'Aurangabad','Total Crimes against Women':1421.0}])], ignore_index=True)

district_wise['District/ Area'].iloc[6] = 'Bid'

district_wise['District/ Area'].iloc[8] = 'Buldana'

district_wise['District/ Area'].iloc[11]  = 'Garhchiroli'

district_wise['District/ Area'].iloc[12]  = 'Gondiya'

district_wise['District/ Area'].iloc[18] = 'Mumbai City'

district_wise['District/ Area'].iloc[19] = 'Mumbai Suburban'

district_wise = pd.concat([district_wise, pd.DataFrame([{'State/ UT': 'Maharashtra', 'District/ Area':'Nagpur','Total Crimes against Women':1754.0}])], ignore_index=True)

district_wise = pd.concat([district_wise, pd.DataFrame([{'State/ UT': 'Maharashtra', 'District/ Area':'Nashik','Total Crimes against Women':1416.0}])], ignore_index=True)

district_wise = pd.concat([district_wise, pd.DataFrame([{'State/ UT': 'Maharashtra', 'District/ Area':'Pune','Total Crimes against Women':1993.0}])], ignore_index=True)

district_wise['District/ Area'].iloc[34] = 'Raigarh'

district_wise = pd.concat([district_wise, pd.DataFrame([{'State/ UT': 'Maharashtra', 'District/ Area':'Thane','Total Crimes against Women':2289.0}])], ignore_index=True)

district_wise = pd.concat([district_wise, pd.DataFrame([{'State/ UT': 'Maharashtra', 'District/ Area':'Solapur','Total Crimes against Women':874.0}])], ignore_index=True)

In [ ]:
# Join both the DataFrames by district names

merged = map_df.set_index('NAME_2').join(district_wise.set_index('District/ Area'))
merged.head()

In [ ]:
# NA Check

merged.isna().sum()

In [ ]:
# summary

merged.describe()

In [ ]:
# create figure and axes for Matplotlib and set the title
fig, ax = plt.subplots(1, figsize=(10, 6))
ax.axis('off')
ax.set_title('District Wise Crime against women in Maharastra in 2015', fontdict={'fontsize': '25', 'fontweight' : '3'})

# plot the figure
merged.plot(column='Total Crimes against Women', cmap='YlOrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

In [ ]:
# Saving the output as a PNG image

fig.savefig("Maharashtra.png", dpi=100)

In [ ]:
# prompt: generate the box plot for all the columns in the dataset Yearwise.xlsx with all columns

# Assuming 'df' is your DataFrame from the previous code

# Create the box plots
for col in df.columns:
  if col not in ['state_name', 'Year']:  # Skip non-numeric columns
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Year', y=col, data=df)
    plt.title(f'Box Plot of {col} over the Years')
    plt.xlabel('Year')
    plt.ylabel(col)
    plt.show()


In [ ]:
# prompt: also generate the heatmap for the same

# Assuming 'df' is your DataFrame from the previous code

# Create the heatmap
plt.figure(figsize=(12, 8))
# Select only numeric columns for correlation calculation
numeric_df = df.select_dtypes(include=['number'])
correlation_matrix = numeric_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Crime Statistics')
plt.show()